# Initialize

To install prerequisites, open a terminal in Jupyter and run the following commands:
```
git clone https://github.com/claudiofahey/p3_test_driver
cd p3_test_driver
pip install -e p3_data
pip install openpyxl
```
Then restart the Python kernel.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import seaborn as sns
import json
from io import StringIO
import importlib
import re

In [ ]:
import p3_data
importlib.reload(p3_data)
from p3_data import (glob_file_list , load_json_from_file, merge_dicts, plot_groups, 
                    get_varying_column_names, filter_dataframe, take_varying_columns,
                    load_json_records_as_dataframe, flatten_multiindex_columns,
                    regex_first_group)

# Load and Clean Results

In [ ]:
# Load result files from P3 Test Driver
src_files = []
src_files += ['../../mnt/isilon/data/genomics/summary/*.json.bz2']
raw_df = load_json_records_as_dataframe(src=src_files, ignore_error=True)

In [ ]:
# Clean raw results
def clean_result(result):
    r = result.copy()
    r['clean'] = False
    try:
        r['utc_begin'] = pd.to_datetime(r['utc_begin'], utc=True)
        r['utc_end'] = pd.to_datetime(r['utc_end'], utc=True)
        r['germline_sec'] = r.germline_result['elapsed_sec']
        r['deepvariant_sec'] = r.deepvariant_result['elapsed_sec']
        r['bwa_mem_sec'] = float(regex_first_group('GPU-BWA Mem time: (.*) seconds', r.germline_result['errors'], search=True))
        r['error'] = r.germline_result['error'] or r.deepvariant_result['error']
        r['num_servers'] = r['num_gpus'] / 16.0  # fraction of server used for this job
        if not r['error']:
            # GPU-BWA Mem + Sorting + MarkingDups + BQSR Generation + BAM writing
            r['bwa_mem_to_bam_writing_sec'] = float(regex_first_group('Processing time: (.*) seconds', r.germline_result['errors'], search=True))
            # Sorting + MarkingDups + BQSR Generation + BAM writing
            r['sorting_to_bam_writing_sec'] = r['bwa_mem_to_bam_writing_sec'] - r['bwa_mem_sec']
            r['haplotypecaller_sec'] = float(regex_first_group('Total time taken: (.*)', r.germline_result['errors'], search=True))
            r['germline_minutes'] = r['germline_sec'] / 60.0
            r['bwa_mem_minutes'] = r['bwa_mem_sec'] / 60.0
            r['bwa_mem_to_bam_writing_minutes'] = r['bwa_mem_to_bam_writing_sec'] / 60.0
            r['sorting_to_bam_writing_minutes'] = r['sorting_to_bam_writing_sec'] / 60.0
            r['haplotypecaller_minutes'] = r['haplotypecaller_sec'] / 60.0
            r['deepvariant_minutes'] = r['deepvariant_sec'] / 60.0
            r['bwa_mem_to_bam_writing_server_minutes'] = r['num_servers'] * r['bwa_mem_to_bam_writing_sec'] / 60.0
            r['haplotypecaller_server_minutes'] = r['num_servers'] * r['haplotypecaller_sec'] / 60.0
            r['deepvariant_server_minutes'] = r['num_servers'] * r['deepvariant_sec'] / 60.0            
            r['fq_file_size_bytes'] = np.sum(r.fq_file_sizes)
            r['fq_file_size_GB'] = r['fq_file_size_bytes'] * 1e-9
            r['bam_file_size_GB'] = r['bam_file_size_bytes'] * 1e-9
            r['haplotypecaller_gvcf_file_size_GB'] = r['haplotypecaller_gvcf_file_size_bytes'] * 1e-9
            r['deepvariant_gvcf_file_size_GB'] = r['deepvariant_gvcf_file_size_bytes'] * 1e-9
        r['clean'] = True
    except Exception as e:
        print('ERROR: %s: %s' % (r['record_uuid'], e))
#         raise e
    return pd.Series(r)

In [ ]:
# r = clean_result(raw_df.iloc[-1])
# pd.DataFrame(r)

In [ ]:
clean1_df = raw_df.apply(clean_result, axis=1)
clean1_df = clean1_df.set_index('record_uuid', drop=False)
clean1_df = clean1_df[clean1_df.clean==True]
clean1_df = clean1_df[clean1_df.error==False]
clean1_df = clean1_df.sort_values(['utc_begin'])

In [ ]:
clean1_df.T

In [ ]:
sample_ids_df = pd.read_csv('sample_ids_300.csv').set_index(['sample_id'])
sample_ids_df.head()

In [ ]:
clean_df = clean1_df.join(sample_ids_df, on=['sample_id'])
clean_df['coverage_binned'] = pd.cut(clean_df['coverage'], bins=[35,40,45,50,51.7,68,80,85], right=False).apply(lambda x: np.round(x.left))
clean_df.head(3).T

# Explore Data

In [ ]:
# Define columns that identify test parameters
param_cols = [
    'sample_id',
    'coverage',
    'num_gpus',
]

In [ ]:
# Define columns that are the output of the experiments
output_cols = [
#     'utc_begin',
    'bwa_mem_to_bam_writing_minutes',
    'haplotypecaller_minutes',
    'deepvariant_minutes',
]

In [ ]:
cols = param_cols + output_cols

In [ ]:
# View most recent results
clean_df[cols].tail(3).T

In [ ]:
clean_df[cols].set_index(['sample_id']).sort_values(['coverage'])

# Export Results

In [ ]:
filt_df = filter_dataframe(
    clean_df,
    batch_uuid=['c9bd8b49-42ec-48de-a80f-99bcd0770964'],
)
len(filt_df)

In [ ]:
filt_df[cols].set_index(['coverage','sample_id','num_gpus']).unstack(['num_gpus'])

In [ ]:
groupby_cols = [
    'coverage_binned',
    'num_gpus',
]
df = filt_df.groupby(groupby_cols).agg({'coverage': ['min','mean','max','size']}).unstack(['num_gpus'])
df.to_excel('results_coverage.xlsx')
df

In [ ]:
groupby_cols = [
    'coverage_binned',
    'num_gpus',
]
agg_cols = [
    'bwa_mem_to_bam_writing_minutes',
    'haplotypecaller_minutes',
    'deepvariant_minutes',
]
df = filt_df[groupby_cols + agg_cols].groupby(groupby_cols).agg(['min','mean','max']).unstack(['num_gpus'])
df.to_excel('results_wall_clock_times.xlsx')
df

In [ ]:
groupby_cols = [
    'coverage_binned',
    'num_gpus',
]
agg_cols = [
    'bwa_mem_to_bam_writing_server_minutes',
    'haplotypecaller_server_minutes',
    'deepvariant_server_minutes',
]
df = filt_df[groupby_cols + agg_cols].groupby(groupby_cols).agg(['min','mean','max']).unstack(['num_gpus'])
df.to_excel('results_server_minutes.xlsx')
df

In [ ]:
groupby_cols = [
    'coverage_binned',
]
agg_cols = [
    'fq_file_size_GB',
    'bam_file_size_GB',
    'haplotypecaller_gvcf_file_size_GB',
    'deepvariant_gvcf_file_size_GB',
]
df = filt_df[groupby_cols + agg_cols].groupby(groupby_cols).agg(['min','mean','max'])
df.to_excel('results_file_sizes.xlsx')
df

In [ ]:
filt_df[['sample_id', 'coverage', 'num_gpus']].sort_values(['coverage'])